In [ ]:
from pypowsybl_jupyter import display_nad
import ipywidgets as widgets
import pypowsybl as pp

In [ ]:
network = pp.network.create_four_substations_node_breaker_network()

In [ ]:
#declare some example callbacks, to simply log the events in a output box
out1 = widgets.Output()
def print_infos(data):   
    with out1:
        print(data)
print_infos('Events log:')

def test_select_node_callback(event):
        print_infos('Selected node: ' + str(event.selected_node))

def test_move_node_callback(event):
        print_infos('Moved node: ' + str(event.moved_node))

def test_move_text_node_callback(event):
        print_infos('Moved text node: ' + str(event.moved_text_node))

In [ ]:
#create a NAD widget for a network and activate the callbacks
nad_widget=display_nad(network.get_network_area_diagram(depth=4), enable_callbacks=True)
nad_widget.on_select_node(test_select_node_callback)
nad_widget.on_move_node(test_move_node_callback)
nad_widget.on_move_text_node(test_move_text_node_callback)

#finally, display the widgets
widgets.HBox([nad_widget, out1])